In [0]:
import csv
import urllib2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
# used for not displaying scientific e notation while using print
np.set_printoptions(precision=8,suppress=True,linewidth=150)


In [0]:
def clean_data(data):
    data.pop(len(data)-1)  # remove last null item in data
    datac = []
    for item in data:
        del item[4]  # remove the class name
        item = np.array(item)
        item = item.astype(np.float)
        datac.append(item)
    return datac


def get_data(url):
    response = urllib2.urlopen(url)
    data_file = csv.reader(response)
    # print(type(data_file))
    data = []
    for row in data_file:
        data.append(row)
    return data

"""
Particles Array
Output: P x C x D
P: Particle count
C: no.of clusters
D: Datapoints second dimension
"""
def init_particles(data, clusters, particles):
    swarm = []
    for i in range(0, particles):
        group = []
        for j in range(0, clusters):
            index = np.random.randint(0, len(data))
            centers = data[index]
            group.append(centers)
        swarm.append(np.array(group))
    return np.array(swarm)

    
  

In [0]:
def distance(data, centers, metric='euclidean'):
    """
    Euclidean distance from each point to each cluster center.
    Parameters
    ----------
    data : 2d array (N x Q)
        Data to be analyzed. There are N data points.
    centers : 2d array (C x Q)
        Cluster centers. There are C clusters, with Q features.
    metric: string
        By default is set to euclidean. Passes any option accepted by
        ``scipy.spatial.distance.cdist``.
    Returns
    -------
    dist : 2d array (N x C)
        Euclidean distance from each point, to each cluster center.
    """
    return cdist(data, centers, metric=metric)

"""
Distance Matrix
Output: P x N x C
P: Particle count
N: no.of Datapoints
C: no.of clusters

"""
def cal_distance_mat(data,particles):
    dist = np.zeros((particles.shape[0], data.shape[0], particles.shape[1]))
    for i in range(0, particles.shape[0]):
        dist[i] = distance(data, particles[i])
        # to avoid dision by zero exception, adding negligible value
        dist[i] = dist[i] + .00000001 
    return dist
        

In [0]:
"""
Membership Matrix
Output: P x N x C
P: Particle count
N: no.of Datapoints
C: no.of clusters

"""
def cal_membership_mat(dist, particles, m=2):
    mem_mat = np.zeros(dist.shape)
    exp = 2/(m-1)
    for p in range(0, particles.shape[0]):
        for i in range(0, dist.shape[1]):
            for j in range(0, dist.shape[2]):
                sum = 0.0
                for k in range(0, dist.shape[2]):
                    div = (dist[p][i][j]/dist[p][i][k])
                    sum = sum + pow(div,exp)
                ans = 1.0/sum
                mem_mat[p][i][j] = ans
    return mem_mat

In [0]:
"""
JMeasure Array
Output: P
P: Particle count

"""
def jmeasure(dist, mem_mat, m=2):
    jm = np.zeros(dist.shape[0])
    for p in range(0,dist.shape[0]):
        jm[p] = 0.0
        for i in range(0, dist.shape[1]):
            sum = 0.0
            for j in range(0, dist.shape[2]):
                sum = sum + (pow(mem_mat[p][i][j], m) * pow(dist[p][i][j], 2))
            jm[p] = jm[p] + sum
    return jm


In [0]:
"""
Centers Matrix
Output: P x C x D
P: Particle count
C: no.of clusters
D: Datapoints second dimension

"""
def update_centers(mem_mat,data,m=2):
    new_centers = np.zeros((mem_mat.shape[0],mem_mat.shape[2],data.shape[1]))
    for i in range(0,mem_mat.shape[0]):
        for j in range(0,mem_mat.shape[2]):
            num = np.zeros((1,data.shape[1]))
            denom = 0.0
            for k in range(0,data.shape[0]):
                p = pow(mem_mat[i][k][j],m)
                denom = denom + p
                num = num + p * data[k]
            new_centers[i][j] = np.array(num/denom)
    return np.array(new_centers)
    
  

In [0]:
def get_labels(cluster_membership):
    labels = []
    for p in cluster_membership:
        labels.append(np.argmax(p))
    return labels

In [8]:
!pip install scikit-fuzzy

import skfuzzy as fuzzy

def lib_fcm(data,cluster_count,m):
    cntr, u, u0, d, jm, p, fpc = fuzzy.cmeans(
        data=data.T, c=cluster_count, m=m, error=0.001, maxiter=1000, init=None, seed=None)
    # u = mem_mat
    return cntr,u.T,jm

    100% |████████████████████████████████| 1.0MB 16.1MB/s 
  Running setup.py bdist_wheel for scikit-fuzzy ... - \ done
  Stored in directory: /root/.cache/pip/wheels/81/d9/eb/b8321c08339f51be3e991e7ae45b9673270cb8ca8271482ab1
Successfully built scikit-fuzzy


In [9]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
m = 2
cluster_count = 3
particle_count = 30

data = get_data(url)
data = clean_data(data)
data = np.array(data)

particles = init_particles(data,cluster_count,particle_count)

print(particles.tolist())

[[[6.7, 2.5, 5.8, 1.8], [4.8, 3.1, 1.6, 0.2], [5.5, 2.5, 4.0, 1.3]], [[4.6, 3.1, 1.5, 0.2], [7.7, 2.8, 6.7, 2.0], [7.0, 3.2, 4.7, 1.4]], [[5.0, 3.4, 1.6, 0.4], [6.3, 2.5, 5.0, 1.9], [4.8, 3.1, 1.6, 0.2]], [[7.2, 3.2, 6.0, 1.8], [7.6, 3.0, 6.6, 2.1], [7.7, 2.6, 6.9, 2.3]], [[6.1, 2.8, 4.0, 1.3], [5.3, 3.7, 1.5, 0.2], [6.4, 2.7, 5.3, 1.9]], [[5.0, 3.0, 1.6, 0.2], [5.3, 3.7, 1.5, 0.2], [6.8, 3.0, 5.5, 2.1]], [[5.6, 2.5, 3.9, 1.1], [4.5, 2.3, 1.3, 0.3], [7.2, 3.6, 6.1, 2.5]], [[5.8, 2.6, 4.0, 1.2], [7.2, 3.0, 5.8, 1.6], [6.3, 2.9, 5.6, 1.8]], [[6.7, 3.1, 4.4, 1.4], [5.0, 3.3, 1.4, 0.2], [6.7, 3.1, 4.7, 1.5]], [[5.6, 2.8, 4.9, 2.0], [5.0, 3.3, 1.4, 0.2], [6.4, 2.8, 5.6, 2.1]], [[4.3, 3.0, 1.1, 0.1], [5.3, 3.7, 1.5, 0.2], [5.0, 3.4, 1.6, 0.4]], [[5.6, 2.7, 4.2, 1.3], [5.2, 3.5, 1.5, 0.2], [6.5, 3.0, 5.8, 2.2]], [[5.7, 2.8, 4.5, 1.3], [6.5, 2.8, 4.6, 1.5], [5.6, 2.8, 4.9, 2.0]], [[4.7, 3.2, 1.3, 0.2], [5.9, 3.2, 4.8, 1.8], [5.1, 3.8, 1.6, 0.2]], [[6.3, 2.8, 5.1, 1.5], [6.8, 3.2, 5.9, 2.3], [5

In [10]:
dist = cal_distance_mat(data,particles)
mem = cal_membership_mat(dist,particles,m)
jmo = jmeasure(dist,mem,m)

for i in range(0,3):
    dist = cal_distance_mat(data,particles)
    mem = cal_membership_mat(dist,particles,m)
    jm = jmeasure(dist,mem,m)
    particles = update_centers(mem,data,m)
    
# get the min JMeasure    
min_jm = np.amin(jm)
# get the index of the min JMeasure particle
min_jm_id = np.where(jm==min_jm)
bp_id = min_jm_id[0][0]

print("Particle(best) "+": "+ str(particles[bp_id].tolist())+"\n" +"JMeasure(old) : "+str(jmo[bp_id])+"\n" +"JMeasure(new) : "+str(jm[bp_id])+"\n")


Particle(best) : [[5.8741401046575685, 2.7557549788345193, 4.3415717529291165, 1.3858011420194036], [5.003260938256684, 3.403454385025158, 1.4837852089229653, 0.25097547381442564], [6.757404935911079, 3.047181686600491, 5.625125686515109, 2.0450723194469496]]
JMeasure(old) : 75.73420018293956
JMeasure(new) : 60.63007138142268



In [11]:
bp_mem = mem[bp_id] 
labels = get_labels(bp_mem)
for i in range(0,len(labels)):
    print(str(data[i].tolist())+"--> "+str(labels[i]))

[5.1, 3.5, 1.4, 0.2]--> 1
[4.9, 3.0, 1.4, 0.2]--> 1
[4.7, 3.2, 1.3, 0.2]--> 1
[4.6, 3.1, 1.5, 0.2]--> 1
[5.0, 3.6, 1.4, 0.2]--> 1
[5.4, 3.9, 1.7, 0.4]--> 1
[4.6, 3.4, 1.4, 0.3]--> 1
[5.0, 3.4, 1.5, 0.2]--> 1
[4.4, 2.9, 1.4, 0.2]--> 1
[4.9, 3.1, 1.5, 0.1]--> 1
[5.4, 3.7, 1.5, 0.2]--> 1
[4.8, 3.4, 1.6, 0.2]--> 1
[4.8, 3.0, 1.4, 0.1]--> 1
[4.3, 3.0, 1.1, 0.1]--> 1
[5.8, 4.0, 1.2, 0.2]--> 1
[5.7, 4.4, 1.5, 0.4]--> 1
[5.4, 3.9, 1.3, 0.4]--> 1
[5.1, 3.5, 1.4, 0.3]--> 1
[5.7, 3.8, 1.7, 0.3]--> 1
[5.1, 3.8, 1.5, 0.3]--> 1
[5.4, 3.4, 1.7, 0.2]--> 1
[5.1, 3.7, 1.5, 0.4]--> 1
[4.6, 3.6, 1.0, 0.2]--> 1
[5.1, 3.3, 1.7, 0.5]--> 1
[4.8, 3.4, 1.9, 0.2]--> 1
[5.0, 3.0, 1.6, 0.2]--> 1
[5.0, 3.4, 1.6, 0.4]--> 1
[5.2, 3.5, 1.5, 0.2]--> 1
[5.2, 3.4, 1.4, 0.2]--> 1
[4.7, 3.2, 1.6, 0.2]--> 1
[4.8, 3.1, 1.6, 0.2]--> 1
[5.4, 3.4, 1.5, 0.4]--> 1
[5.2, 4.1, 1.5, 0.1]--> 1
[5.5, 4.2, 1.4, 0.2]--> 1
[4.9, 3.1, 1.5, 0.1]--> 1
[5.0, 3.2, 1.2, 0.2]--> 1
[5.5, 3.5, 1.3, 0.2]--> 1
[4.9, 3.1, 1.5, 0.1]--> 1
[4.4, 3.0, 1

In [0]:
import numpy as np
import random

# Calculates distance, membership matrices and JMeasure array
def calc_req(data,particles,m=2):
    dist = cal_distance_mat(data,particles)
    mem_mat = cal_membership_mat(dist,particles,m)
    jm = jmeasure(dist,mem_mat,m)
    return dist,mem_mat,jm

def calc_inertia_psok(wmax,wmin,i,imax):
    w = wmax-(i*((wmax-wmin)/imax))
    return w

def calc_c(c1i,c1f,c2i,c2f,i,imax):
    # c1i > c2i & c1f < c2f
    c1 = c1i + ((c1f-c1i)/imax)*i
    c2 = c2i + ((c2f-c2i)/imax)*i
    return c1,c2
    

class Particle:
    def __init__(self,x0):
        global num_dimensions
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual
        
        num_dimensions = len(x0)
        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self,jm):
        self.err_i= jm

        # check to see if the current position is an individual best
        if self.err_i<self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i
                    
    # update new particle velocity
    def update_velocity(self,pos_best_g,i,imax):
#         w=0.72       # constant inertia weight (how much to weigh the previous velocity)
#         c1=1.49        # cognative constant
#         c2=1.49        # social constant
        
        wmax = 0.9
        wmin = 0.4
        c1i = 2.5
        c1f = 0.5
        c2i = 0.5
        c2f = 2.5
        
        w = calc_inertia_psok(wmax,wmin,i,imax)
        c1,c2 = calc_c(c1i,c1f,c2i,c2f,i,imax)
        
        for i in range(0,num_dimensions):
            r1=random.uniform(0,1)
            r2=random.uniform(0,1)
            
            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds=None):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]
            
            if(bounds):
                # adjust maximum position if necessary
                if self.position_i[i]>bounds[i][1]:
                    self.position_i[i]=bounds[i][1]

                # adjust minimum position if neseccary
                if self.position_i[i]<bounds[i][0]:
                    self.position_i[i]=bounds[i][0]

In [0]:
def PSO(data, particles0, maxiter, m=2, bounds=None):

    num_particles = len(particles0)
    err_best_g = -1                   # best error for group
    pos_best_g = []                   # best position for group

    # establish the swarm
    swarm = []
    for i in range(0, num_particles):
        swarm.append(Particle(particles0[i]))

    # begin optimization loop
    for i in range(0,maxiter):
        
#         prev_err_best_g = err_best_g

        particles = []
        # calculate distance, membership matrix
        for k in range(0, num_particles):
            particles.append(swarm[k].position_i)
        particles = np.array(particles)

        dist, mem_mat, jm = calc_req(data, particles, m)

        # print i,err_best_g
        # cycle through particles in swarm and evaluate fitness
        for j in range(0, num_particles):
            swarm[j].evaluate(jm[j])

            # determine if current particle is the best (globally)
            if swarm[j].err_i < err_best_g or err_best_g == -1:
                pos_best_g = list(swarm[j].position_i)
                err_best_g = float(swarm[j].err_i)

        # cycle through swarm and update velocities and position
        # i, maxiter are req for calculation of dyanamic learning values
        for j in range(0, num_particles):
            swarm[j].update_velocity(pos_best_g,i,maxiter)
            swarm[j].update_position(bounds)
        
        print("Iteration: " + str(i+1) + "/" + str(maxiter) +
            " " + str(pos_best_g)+","+str(err_best_g))
        
#         if(prev_err_best_g == err_best_g and i>maxiter/25):
#             break

    # print final results
    print('FINAL:')
    print(pos_best_g)
    print(err_best_g)
    
    return pos_best_g

In [14]:
best_particle = PSO(data,particles,maxiter=1000,m=2)

Iteration: 1/1000 [array([5.87414455, 2.75539251, 4.34451632, 1.38865484]), array([5.00280632, 3.40311838, 1.48349256, 0.25084286]), array([6.77364659, 3.05100284, 5.64237923, 2.04944073])],60.5896518461
Iteration: 2/1000 [array([5.87414455, 2.75539251, 4.34451632, 1.38865484]), array([5.00280632, 3.40311838, 1.48349256, 0.25084286]), array([6.77364659, 3.05100284, 5.64237923, 2.04944073])],60.5896518461
Iteration: 3/1000 [array([5.87414455, 2.75539251, 4.34451632, 1.38865484]), array([5.00280632, 3.40311838, 1.48349256, 0.25084286]), array([6.77364659, 3.05100284, 5.64237923, 2.04944073])],60.5896518461
Iteration: 4/1000 [array([5.87414455, 2.75539251, 4.34451632, 1.38865484]), array([5.00280632, 3.40311838, 1.48349256, 0.25084286]), array([6.77364659, 3.05100284, 5.64237923, 2.04944073])],60.5896518461
Iteration: 5/1000 [array([5.87414455, 2.75539251, 4.34451632, 1.38865484]), array([5.00280632, 3.40311838, 1.48349256, 0.25084286]), array([6.77364659, 3.05100284, 5.64237923, 2.049440

In [15]:
# partcles matrix is of shape P x C x D
bp = []
bp.append(best_particle)
bp = np.array(bp)
dist = cal_distance_mat(data,bp)
mem_mat = cal_membership_mat(dist,bp,m)
labels = get_labels(mem_mat[0])
for i in range(0,len(labels)):
    print(str(data[i].tolist())+"--> "+str(labels[i]))

[5.1, 3.5, 1.4, 0.2]--> 1
[4.9, 3.0, 1.4, 0.2]--> 1
[4.7, 3.2, 1.3, 0.2]--> 1
[4.6, 3.1, 1.5, 0.2]--> 1
[5.0, 3.6, 1.4, 0.2]--> 1
[5.4, 3.9, 1.7, 0.4]--> 1
[4.6, 3.4, 1.4, 0.3]--> 1
[5.0, 3.4, 1.5, 0.2]--> 1
[4.4, 2.9, 1.4, 0.2]--> 1
[4.9, 3.1, 1.5, 0.1]--> 1
[5.4, 3.7, 1.5, 0.2]--> 1
[4.8, 3.4, 1.6, 0.2]--> 1
[4.8, 3.0, 1.4, 0.1]--> 1
[4.3, 3.0, 1.1, 0.1]--> 1
[5.8, 4.0, 1.2, 0.2]--> 1
[5.7, 4.4, 1.5, 0.4]--> 1
[5.4, 3.9, 1.3, 0.4]--> 1
[5.1, 3.5, 1.4, 0.3]--> 1
[5.7, 3.8, 1.7, 0.3]--> 1
[5.1, 3.8, 1.5, 0.3]--> 1
[5.4, 3.4, 1.7, 0.2]--> 1
[5.1, 3.7, 1.5, 0.4]--> 1
[4.6, 3.6, 1.0, 0.2]--> 1
[5.1, 3.3, 1.7, 0.5]--> 1
[4.8, 3.4, 1.9, 0.2]--> 1
[5.0, 3.0, 1.6, 0.2]--> 1
[5.0, 3.4, 1.6, 0.4]--> 1
[5.2, 3.5, 1.5, 0.2]--> 1
[5.2, 3.4, 1.4, 0.2]--> 1
[4.7, 3.2, 1.6, 0.2]--> 1
[4.8, 3.1, 1.6, 0.2]--> 1
[5.4, 3.4, 1.5, 0.4]--> 1
[5.2, 4.1, 1.5, 0.1]--> 1
[5.5, 4.2, 1.4, 0.2]--> 1
[4.9, 3.1, 1.5, 0.1]--> 1
[5.0, 3.2, 1.2, 0.2]--> 1
[5.5, 3.5, 1.3, 0.2]--> 1
[4.9, 3.1, 1.5, 0.1]--> 1
[4.4, 3.0, 1